In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

base_url = "https://www.yellowpages-uae.com/uae/restaurant"
all_data = []

def get_total_pages(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    pagination = soup.find('div', id='pagination')
    if pagination:
        pages_div = pagination.find('div', class_='pages')
        if pages_div:
            page_links = pages_div.find_all('a')
            total_pages = len(page_links) + 1  
            return total_pages
    return 1 
 
# Function to extract data from each page
def scrape_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    detail = soup.findAll(attrs={'class': 'row box foc'})

    for i in detail:
        row_data = {}
        info = i.h2.get_text(strip=True)
        row_data['Name'] = info

        address_span = i.find('span', itemprop='streetAddress')
        if address_span:
            address = address_span.get_text(strip=True)
        else:
            address = None
        row_data['Location'] = address

        c = i.find('strong', itemprop="addressLocality").get_text(strip=True)
        row_data['City'] = c

        r = i.find('strong', itemprop="addressRegion").get_text(strip=True)
        row_data['Region'] = r

        p = i.find('span', itemprop="postalCode", class_="pobox")
        if p:
            post = p.get_text(strip=True)
        else:
            post = None
        row_data['P.O. Box'] = post

        ph = i.find('a', title="Phone")
        if ph:
            p = ph.get_text(strip=True)
            p_h = p.replace("Call", "").strip()
        else:
            p_h = None
        row_data['Phone'] = p_h

        mo = i.find('a', title="Mobile")
        if mo:
            my = mo.get_text(strip=True)
            mob = my.replace("Call", "").strip()
        else:
            mob = None
        row_data['Mobile'] = mob

        l = i.find('a', class_="website")
        if l:
            url = l["href"]
        else:
            url = None
        row_data['Company page link'] = url

        lo = i.find('img', itemprop="image")
        if lo and 'data-src' in lo.attrs:
            src = lo["data-src"]
        else:
            src = None
        row_data['Logo URL'] = src

        all_data.append(row_data)

# Get total pages
total_pages = get_total_pages(base_url)

# Loop through each page until the total number of entries reaches 1317
while len(all_data) < 1317:
    url = f"{base_url}-{len(all_data)}.html"  # Each page contains 20 entries
    scrape_page(url)

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(all_data)

# Save DataFrame to CSV file
df.to_csv('results.csv', index=False)
print("Scraping completed and data saved to restaurant_data.csv")


/tmp/ipykernel_274864/1695470321.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Scraping completed and data saved to restaurant_data.csv
